Setting PyGithub

In [1]:
from github import Github

g = Github("your token")

In [7]:
languages = ["JavaScript", "Python", "Go", "Java", "C#", "C++", "C", "TypeScript", "Ruby", "PHP"]

In [2]:
popular_repos = g.search_repositories("stars:>5000")
popular_repos.totalCount

1000

In [3]:

print(len(list(popular_repos)))

RateLimitExceededException: 403 {'message': 'API rate limit exceeded for user ID 5610771.', 'documentation_url': 'https://developer.github.com/v3/#rate-limiting'}

In [62]:
resp = []
for p in popular_repos:
    if (not p.language in languages):
        resp.append(p.language)

s = list(set(resp))
s

['Dockerfile',
 'Scala',
 None,
 'Julia',
 'HTML',
 'CoffeeScript',
 'OCaml',
 'Elixir',
 'Haskell',
 'Perl',
 'CSS',
 'TeX',
 'Swift',
 'Rust',
 'Clojure',
 'Kotlin',
 'Lua',
 'Assembly',
 'Emacs Lisp',
 'Batchfile',
 'Objective-C++',
 'Jupyter Notebook',
 'C',
 'Vue',
 'Objective-C',
 'Makefile',
 'Dart',
 'C++',
 'Shell',
 'Crystal',
 'Rascal',
 'Vim script']

In [12]:
popular_repos

In [13]:
import requests
import pandas as pd
from pandas.io.json import json_normalize
languages = ["C"]
def get_test_coverage_codecov(full_name):
    try:
        resp = requests.get('https://codecov.io/api/gh/{0}'.format(full_name))
        df = json_normalize(resp.json()['commits'])
        df = df[df['timestamp'] == df['timestamp'].max()]
        data =[]
        data.append(df['totals.c'][0])
        data.append(df['timestamp'][0])
        return data[0], data[1]
    except:
        return None,None
    
def get_test_coverage_coveralls(full_name):
    try:
        resp = requests.get('https://coveralls.io/github/{0}.json'.format(full_name))
        data = []
        data.append(resp.json()['covered_percent'])
        timestamp = resp.json()['created_at']
        timestamp = timestamp.replace("T"," ")
        timestamp = timestamp.replace("Z", " ")
        data.append(timestamp)
        return data[0], data[1]
    except:
        return None,None

# get_test_coverage('scikit-learn/scikit-learn')

In [116]:
cov = get_test_coverage_codecov("scikit-learn/scikit-learn")
print(cov)
cov2 = get_test_coverage_coveralls("scikit-learn/scikit-learn")
print(cov2)


('96.76506', '2019-07-11 21:30:31')
(64.2930432998153, '2019-06-25 19:05:57 ')


Iterates over the repositories searching for coverage data

In [5]:
for lang in languages:
    popular_repos = g.search_repositories("stars:>5000 is:public language:{0}".format(lang))
    print(lang+": "+str(popular_repos.totalCount))

JavaScript: 933
Python: 320
Go: 188
Java: 305
C#: 42
C++: 140
C: 93
TypeScript: 118
Ruby: 83
PHP: 125


In [14]:
repos = []
for lang in languages:
    popular_repos = g.search_repositories("stars:>5000 is:public language:{0}".format(lang))
    print("Collecting data for "+str(lang)+". "+str(popular_repos.totalCount)+" repositories in total.")
    iteration = 1
    inter = 1
    for r in popular_repos:
        codecov,date_codecov = get_test_coverage_codecov(r.full_name)
        coveralls,date_coveralls = get_test_coverage_coveralls(r.full_name)
        if(inter==10):
            print(str(iteration)+" of "+str(popular_repos.totalCount))
            inter=0
        #print(r.full_name+" codecov: "+str(cov_codecov)+" coveralls: "+str(cov_coveralls))
        if(date_codecov != None or date_coveralls != None):
            repo = {
                'id': r.id,
                'name': r.name,
                'full_name': r.full_name,
                'stargazers_count': r.stargazers_count,
                'open_issues_count': r.open_issues_count,
                'language': r.language,
                'size': r.size,
                'test_coverage_codecov': codecov,
                'test_coverage_coveralls': coveralls,
                'date_codecov': date_codecov,
                'date_coveralls': date_coveralls
            }
            repos.append(repo)
        iteration+=1
        inter+=1

10 of 93
20 of 93
30 of 93
40 of 93
50 of 93
60 of 93
70 of 93
80 of 93
90 of 93


In [15]:
import pandas as pd

df = pd.DataFrame(repos)
df

,date_codecov,date_coveralls,full_name,id,language,name,open_issues_count,size,stargazers_count,test_coverage_codecov,test_coverage_coveralls
0,None,2019-03-05 19:24:58,php/php-src,1903522,C,php-src,160,369944,23961,None,92.319811
1,None,2019-07-09 23:27:45,stedolan/jq,5101141,C,jq,487,6626,14730,None,75.260888
2,2017-08-18 10:17:25,2019-07-12 16:16:45,curl/curl,569041,C,curl,65,63571,14084,0,77.103155
3,2019-07-12 04:53:07,None,numpy/numpy,908607,C,numpy,1946,68971,11198,85.71074,NaN
4,None,2019-07-08 23:58:30,openssl/openssl,7634677,C,openssl,949,173437,10838,None,57.640976
5,2019-07-12 16:16:44,2019-04-29 17:04:34,radare/radare2,4869294,C,radare2,1253,106398,10608,37.78201,41.148886
6,2017-05-14 05:33:22,None,esp8266/Arduino,32969220,C,Arduino,442,75138,9692,27.57866,NaN
7,None,2019-07-12 07:22:46,micropython/micropython,15337142,C,micropython,725,41224,8710,None,97.997557
8,2019-07-09 22:20:36,None,timescale/timescaledb,84240850,C,timescaledb,119,6808,7293,91.25943,NaN
9,None,2019-06-21 21:52:55,jedisct1/libsodium,7710647,C,libsodium,2,7548,7182,None,97.166470


Total amount of repositories with more than 5 thousand stars

In [16]:
len(repos)


11

In [17]:
df = df.drop_duplicates(subset=['full_name'])

In [18]:
len(df)

11

Total amount of repositories with coverage data from codecov

In [19]:
len(df[df['test_coverage_codecov'].notnull()])

6

In [20]:

len(df[df['test_coverage_coveralls'].notnull()])

7

In [21]:
aux = df[df['test_coverage_codecov'].notnull()]

len(aux[aux['test_coverage_coveralls'].notnull()])

2

for r in repos:
    if(r['full_name'] == "selectize/selectize.js"):
        print("language"+r['language'])
        print("stars"+str(r['stargazers_count']))
        print("issues"+str(r['open_issues_count']))
        print("size"+str(r['size']))
        

Saving in CSV

In [22]:
df.to_csv('repos_coverage_c.csv')